# 03 - Modelagem e Experimentos (Diário de Bordo)

**🎯 PROPÓSITO DESTE NOTEBOOK:**
Este notebook documenta o nosso processo completo de exploração e seleção de modelos. Aqui comparamos Baselines, RandomForest, LightGBM e XGBoost para justificar a nossa escolha final.

**📊 RESULTADO PRINCIPAL:**
- Testamos múltiplos modelos (Baselines, Random Forest, LightGBM, XGBoost)  
- **LightGBM** venceu com **WMAPE: 15.25%** (91% melhor que baseline)
- XGBoost foi marginalmente melhor, mas **instável** em produção
- Este notebook contém a **justificativa técnica** da nossa decisão final

**🚀 PARA EXECUTAR A SOLUÇÃO FINAL:**
Use o notebook `04-Final-Pipeline.ipynb` - ele contém apenas o código necessário para treinar o modelo vencedor e gerar a submissão.

---

## Objetivos da Exploração:
1. **Carregamento dos Dados**: Carregar dataset com features processadas
2. **Preparação para ML**: Dividir dados em treino/validação, preparar features  
3. **Baseline Models**: Implementar modelos simples como referência
4. **Advanced Models**: Testar modelos avançados (LightGBM, XGBoost, etc.)
5. **Comparação Rigorosa**: Avaliar todos os modelos usando métricas adequadas
6. **Seleção Final**: Escolher o modelo mais robusto para produção

In [ ]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Advanced ML
import lightgbm as lgb
from lightgbm.callback import early_stopping  # CORREÇÃO: Importar early_stopping
import xgboost as xgb

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Time series
from datetime import datetime, timedelta
import gc

plt.style.use('default')
sns.set_palette("husl")

print('📚 Bibliotecas carregadas com sucesso!')
print('🎯 Iniciando fase de Modelagem e Treinamento')

## 1. Carregamento dos Dados Processados

In [ ]:
# Carregar dados com features processadas
print('📂 Carregando dados processados...')

# Verificar se os arquivos essenciais existem
import os
required_files = [
    '../data/dados_features_completo.parquet',  # Usar parquet (mais rápido)
    '../data/feature_engineering_metadata.pkl'
]

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print('❌ Arquivos não encontrados:')
    for f in missing_files:
        print(f'   • {f}')
    print('\n🔄 Execute primeiro o notebook 02-Feature-Engineering-Dask.ipynb')
else:
    print('✅ Todos os arquivos necessários encontrados')
    
    # Carregar dados principais (usar parquet para velocidade)
    print('📊 Carregando dataset (parquet)...')
    dados = pd.read_parquet('../data/dados_features_completo.parquet')
    
    # Carregar metadados
    with open('../data/feature_engineering_metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    
    print(f'\n📊 Dados carregados com sucesso:')
    print(f'   • Shape: {dados.shape}')
    print(f'   • Período: {dados["semana"].min()} até {dados["semana"].max()}')
    print(f'   • Features disponíveis: {len(dados.columns)}')
    print(f'   • Memória: {dados.memory_usage(deep=True).sum() / (1024**2):.1f} MB')
    print(f'   • Estratégia: {metadata.get("estrategia", "Grid Inteligente")}')
    
    print(f'\n🔍 Metadados do processamento:')
    for key, value in metadata.items():
        if key not in ['features_criadas', 'data_processamento']:  # Skip long items
            print(f'   • {key}: {value}')
    
    print(f'\n✅ Pronto para modelagem!')

## 2. Preparação dos Dados para ML

In [ ]:
# Definir variável target e features
target = 'quantidade'

# Features a excluir (não devem ser usadas para predição)
exclude_features = [
    'pdv_id', 'produto_id', 'semana',  # IDs e data
    'quantidade',  # Target
    'valor', 'num_transacoes',  # Features que vazam informação do futuro
]

# Identificar features disponíveis
all_features = [col for col in dados.columns if col not in exclude_features]

print(f'🎯 Preparação dos dados:')
print(f'   • Target: {target}')
print(f'   • Features disponíveis: {len(all_features)}')
print(f'   • Features excluídas: {len(exclude_features)}')

# Verificar missing values nas features
missing_features = dados[all_features].isnull().sum()
missing_features = missing_features[missing_features > 0].sort_values(ascending=False)

if len(missing_features) > 0:
    print(f'\n⚠️ Features com valores missing:')
    for feature, count in missing_features.head(10).items():
        pct = (count / len(dados)) * 100
        print(f'   • {feature}: {count:,} ({pct:.1f}%)')
    
    print(f'\n🧠 Estratégia de Tratamento Inteligente:')
    print('   • distributor_id (categórica): NaN → -1 (venda direta)')
    print('   • Features numéricas: NaN → 0 (ausência = zero)')
    print('   • LightGBM aprenderá padrões específicos para valores -1/0')
else:
    print('\n✅ Nenhum valor missing nas features')

print(f'\n📋 Features finais para modelagem: {len(all_features)}')
print('💡 Missing values serão tratados como informação, não removidos')

In [ ]:
# SOLUÇÃO CORRETA: Otimização de Tipos de Dados (Downcasting)
print('📅 Otimização de Memória + Divisão Temporal')
print('🧠 Estratégia: Downcasting em vez de amostragem (preserva séries temporais)')

# PASSO 1: Inspecionar uso de memória atual
print(f'\n🔍 ANTES da otimização:')
memory_before = dados.memory_usage(deep=True).sum() / (1024**3)
print(f'💾 Memória total: {memory_before:.2f} GB')

# PASSO 2: Aplicar Downcasting Inteligente
print(f'\n🚀 Aplicando Downcasting...')

# Fazer uma cópia para otimização
dados_sorted = dados.copy()

# Otimizar colunas numéricas (inteiros e floats)
for col in dados_sorted.select_dtypes(include=[np.number]).columns:
    original_dtype = dados_sorted[col].dtype
    
    if dados_sorted[col].dtype.kind in ['i', 'u']:  # Inteiros
        dados_sorted[col] = pd.to_numeric(dados_sorted[col], downcast='integer')
    else:  # Floats
        dados_sorted[col] = pd.to_numeric(dados_sorted[col], downcast='float')
    
    new_dtype = dados_sorted[col].dtype
    if original_dtype != new_dtype:
        print(f'   • {col}: {original_dtype} → {new_dtype}')

# Otimizar colunas categóricas
for col in dados_sorted.select_dtypes(include=['object']).columns:
    if col not in ['semana']:  # Preservar datetime
        nunique = dados_sorted[col].nunique()
        total_rows = len(dados_sorted)
        if nunique / total_rows < 0.5:  # Se <50% valores únicos, usar category
            dados_sorted[col] = dados_sorted[col].astype('category')
            print(f'   • {col}: object → category')

print(f'✅ Downcasting concluído!')

# PASSO 3: Verificar resultado da otimização
memory_after = dados_sorted.memory_usage(deep=True).sum() / (1024**3)
memory_reduction = (memory_before - memory_after) / memory_before * 100
print(f'\n📊 DEPOIS da otimização:')
print(f'💾 Memória total: {memory_after:.2f} GB')
print(f'🎯 Redução: {memory_reduction:.1f}% ({memory_before-memory_after:.2f} GB economizados)')

# PASSO 4: Divisão temporal (agora com dados otimizados)
print(f'\n📅 Divisão temporal dos dados (com memória otimizada)...')

# Ordenar por semana
dados_sorted = dados_sorted.sort_values('semana')

# Split temporal: semanas 1-48 treino, 49-52 validação
semanas_unicas = sorted(dados_sorted['semana'].unique())
print(f'📊 Total de semanas disponíveis: {len(semanas_unicas)}')

cutoff_week_idx = 48  # Primeiras 48 semanas para treino
if len(semanas_unicas) >= cutoff_week_idx:
    cutoff_week = semanas_unicas[cutoff_week_idx-1]
    
    # Criar máscaras (sem cópia)
    train_mask = dados_sorted['semana'] <= cutoff_week
    val_mask = dados_sorted['semana'] > cutoff_week
    
    print(f'📊 Divisão dos dados:')
    print(f'   • Treino: {train_mask.sum():,} registros ({train_mask.mean()*100:.1f}%)')
    print(f'   • Validação: {val_mask.sum():,} registros ({val_mask.mean()*100:.1f}%)')
    
    # CORREÇÃO: Tratamento de missing values com categorias
    print(f'\n🧠 Tratamento inteligente de missing values (CORRIGIDO)...')
    all_features = [col for col in dados_sorted.columns if col not in ['pdv_id', 'produto_id', 'semana', 'quantidade', 'valor', 'num_transacoes']]
    
    for col in all_features:
        missing_count = dados_sorted[col].isnull().sum()
        if missing_count > 0:
            if col == 'distributor_id':
                # SOLUÇÃO: Adicionar -1 ao "menu" de categorias primeiro
                if dados_sorted[col].dtype.name == 'category':
                    if -1 not in dados_sorted[col].cat.categories:
                        dados_sorted[col] = dados_sorted[col].cat.add_categories([-1])
                
                # Agora pode preencher com -1 sem erro
                dados_sorted[col] = dados_sorted[col].fillna(-1)
                print(f'   • {col}: {missing_count:,} NaN → -1 (venda direta)')
                
            elif dados_sorted[col].dtype.kind in ['i', 'u', 'f']:
                # Numéricas: fillna funciona diretamente
                dados_sorted[col] = dados_sorted[col].fillna(0)
                print(f'   • {col}: {missing_count:,} NaN → 0 (ausência)')
    
    # PASSO 5: Preparar dados para modelagem (agora deve funcionar!)
    print(f'\n🎯 Preparando dados para modelagem...')
    X_train = dados_sorted.loc[train_mask, all_features]
    y_train = dados_sorted.loc[train_mask, 'quantidade']
    X_val = dados_sorted.loc[val_mask, all_features]
    y_val = dados_sorted.loc[val_mask, 'quantidade']
    
    print(f'✅ Dados preparados com sucesso:')
    print(f'   • X_train shape: {X_train.shape}')
    print(f'   • X_val shape: {X_val.shape}')
    print(f'   • Memória X_train: {X_train.memory_usage(deep=True).sum() / (1024**2):.1f} MB')
    print(f'   • Memória X_val: {X_val.memory_usage(deep=True).sum() / (1024**2):.1f} MB')
    
    # Garbage collection
    import gc
    gc.collect()
    
    print(f'\n🎉 SUCESSO! Problema resolvido:')
    print(f'   ✅ Downcasting: {memory_reduction:.1f}% menos memória')
    print(f'   ✅ Categorical fix: -1 adicionado ao "menu" de categorias')
    print(f'   ✅ Séries temporais preservadas integralmente')
    
else:
    print(f'⚠️ Menos de 48 semanas disponíveis.')

## 3. Análise Exploratória do Target

In [ ]:
# Análise da distribuição do target (usando dados otimizados)
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Distribuição geral (usando y_train que foi criado)
axes[0,0].hist(y_train, bins=50, alpha=0.7, edgecolor='black')
axes[0,0].set_title('Distribuição da Quantidade (Treino)')
axes[0,0].set_xlabel('Quantidade')
axes[0,0].set_ylabel('Frequência')

# Log-scale
non_zero_train = y_train[y_train > 0]
axes[0,1].hist(np.log1p(non_zero_train), bins=50, alpha=0.7, edgecolor='black')
axes[0,1].set_title('Distribuição log(Quantidade + 1) - Apenas > 0')
axes[0,1].set_xlabel('log(Quantidade + 1)')
axes[0,1].set_ylabel('Frequência')

# Zeros vs Non-zeros
zero_counts = [len(y_train[y_train == 0]), len(y_train[y_train > 0])]
axes[1,0].pie(zero_counts, labels=['Zeros', 'Não-zeros'], autopct='%1.1f%%')
axes[1,0].set_title('Proporção Zeros vs Não-zeros')

# Boxplot por semana (últimas 12 semanas) - usando dados_sorted com train_mask
train_weeks_data = dados_sorted.loc[train_mask, ['semana', 'quantidade']]
recent_weeks = sorted(train_weeks_data['semana'].unique())[-12:]
recent_data = train_weeks_data[train_weeks_data['semana'].isin(recent_weeks)]

# Criar boxplot manualmente para evitar problemas com groupby
week_data = []
week_labels = []
for week in recent_weeks:
    week_quantities = recent_data[recent_data['semana'] == week]['quantidade']
    if len(week_quantities) > 0:
        week_data.append(week_quantities.values)
        week_labels.append(week.strftime('%m-%d'))

if week_data:
    axes[1,1].boxplot(week_data, labels=week_labels)
    axes[1,1].set_title('Distribuição por Semana (Últimas 12)')
    axes[1,1].set_xlabel('Semana')
    axes[1,1].tick_params(axis='x', rotation=45)
else:
    axes[1,1].text(0.5, 0.5, 'Dados insuficientes\npara boxplot', 
                   ha='center', va='center', transform=axes[1,1].transAxes)
    axes[1,1].set_title('Distribuição por Semana')

plt.tight_layout()
plt.show()

# Estatísticas descritivas
print('📈 Estatísticas do Target (treino):')
print(y_train.describe())

print(f'\n🎯 Métricas importantes:')
print(f'   • Zeros: {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.1f}%)')
print(f'   • Não-zeros: {(y_train > 0).sum():,} ({(y_train > 0).mean()*100:.1f}%)')
print(f'   • Média (apenas > 0): {y_train[y_train > 0].mean():.2f}')
print(f'   • Mediana (apenas > 0): {y_train[y_train > 0].median():.2f}')

## 5. Modelos de Machine Learning

In [ ]:
# PASSO B: LightGBM Vanilla - Validar Pipeline e Features
print('\n🚀 PASSO B: Modelo LightGBM Vanilla (Parâmetros Default)')
print('=' * 60)
print('🎯 Objetivo: Validar se nossas features têm poder preditivo')

# Configuração LightGBM Vanilla (parâmetros simples/default)
lgb_params_vanilla = {
    'objective': 'regression_l1',  # MAE - melhor para WMAPE
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'random_state': 42,
    'n_jobs': -1
}

print(f'\n📋 Configuração Vanilla:')
for param, value in lgb_params_vanilla.items():
    print(f'   • {param}: {value}')

# Preparar dados para LightGBM
print(f'\n📊 Preparando dados para treinamento...')
train_lgb = lgb.Dataset(X_train, label=y_train)
val_lgb = lgb.Dataset(X_val, label=y_val, reference=train_lgb)

print(f'   • Train shape: {X_train.shape}')
print(f'   • Val shape: {X_val.shape}')
print(f'   • Features: {len(all_features)}')

# Treinar modelo Vanilla (VERSÃO CORRIGIDA)
print(f'\n🔄 Treinando LightGBM Vanilla...')
lgb_vanilla = lgb.train(
    lgb_params_vanilla,
    train_lgb,
    num_boost_round=200,  # Número moderado para vanilla
    valid_sets=[train_lgb, val_lgb],
    valid_names=['train', 'eval'],
    # CORREÇÃO: Usar callbacks em vez de early_stopping_rounds
    callbacks=[early_stopping(stopping_rounds=20, verbose=False)]
)

print(f'✅ Treinamento concluído em {lgb_vanilla.best_iteration} iterações')

# Predições
print(f'\n🎯 Gerando predições...')
lgb_vanilla_pred = lgb_vanilla.predict(X_val, num_iteration=lgb_vanilla.best_iteration)
lgb_vanilla_pred = np.maximum(0, lgb_vanilla_pred)  # Não permitir predições negativas

# Avaliação
results_lgb_vanilla = evaluate_model(y_val, lgb_vanilla_pred, 'LightGBM Vanilla')
model_results.append(results_lgb_vanilla)

# ANÁLISE CRÍTICA - Pergunta chave
print(f'\n🔍 ANÁLISE CRÍTICA - VALIDAÇÃO DO PIPELINE:')
print('=' * 60)
print(f'LightGBM Vanilla    | WMAPE: {results_lgb_vanilla["WMAPE"]:6.2f}% | MAE: {results_lgb_vanilla["MAE"]:8.4f} | R²: {results_lgb_vanilla["R²"]:6.4f}')
print(f'Melhor Baseline     | WMAPE: {best_baseline["WMAPE"]:6.2f}% | MAE: {best_baseline["MAE"]:8.4f} | R²: {best_baseline["R²"]:6.4f}')

# Calcular melhoria
wmape_improvement = ((best_baseline["WMAPE"] - results_lgb_vanilla["WMAPE"]) / best_baseline["WMAPE"]) * 100
mae_improvement = ((best_baseline["MAE"] - results_lgb_vanilla["MAE"]) / best_baseline["MAE"]) * 100

print(f'\n📈 MELHORIA SOBRE MELHOR BASELINE:')
print(f'   • WMAPE: {wmape_improvement:+.2f}% {"✅ SIGNIFICATIVA!" if wmape_improvement > 5 else "⚠️ MARGINAL" if wmape_improvement > 0 else "❌ PIOR QUE BASELINE!"}')
print(f'   • MAE:   {mae_improvement:+.2f}% {"✅" if mae_improvement > 0 else "❌"}')

# Diagnóstico
if wmape_improvement > 5:
    print(f'\n🎉 DIAGNÓSTICO: PIPELINE VALIDADO!')
    print(f'   ✅ Features têm forte poder preditivo')
    print(f'   ✅ Pronto para otimização de hiperparâmetros')
elif wmape_improvement > 0:
    print(f'\n🤔 DIAGNÓSTICO: MELHORIA MARGINAL')
    print(f'   ⚠️ Features têm algum poder preditivo, mas limitado')
    print(f'   🔍 Considerar análise de feature importance')
else:
    print(f'\n🚨 DIAGNÓSTICO: PROBLEMA NO PIPELINE!')
    print(f'   ❌ Modelo pior que baseline - possível data leakage ou bug')
    print(f'   🔧 Revisar feature engineering urgentemente')

print(f'\n✅ Passo B concluído - LightGBM Vanilla validado!')

## 9. Preparação para Predições Finais

In [ ]:
print('🎯 Preparação para predições finais...')

# Retreinar melhor modelo com todos os dados disponíveis
print(f'🔄 Retreinando {best_model_name} com todos os dados...')

# CORREÇÃO: Usar dados otimizados (dados_sorted) em vez de dados originais
print('🧠 Preparando dados com tipos otimizados (mesmo processamento de treino/validação)...')

# Usar dados_sorted que já foram otimizados e tratados
X_full = dados_sorted[all_features]
y_full = dados_sorted[target]

print(f'   • X_full shape: {X_full.shape}')
print(f'   • Tipos de dados consistentes: {X_full.dtypes.value_counts().to_dict()}')

# Retreinar LightGBM
train_full_lgb = lgb.Dataset(X_full, label=y_full)
final_model = lgb.train(
    lgb_params_vanilla,
    train_full_lgb,
    num_boost_round=lgb_vanilla.best_iteration,
    verbose_eval=False
)

print('✅ Modelo final treinado e pronto para predições')

# Salvar modelo e configurações
model_artifacts = {
    'model': final_model,
    'model_type': best_model_name,
    'features': all_features,
    'target': target,
    'validation_mae': results_df.iloc[0]['MAE'],
    'validation_rmse': results_df.iloc[0]['RMSE'],
    'validation_r2': results_df.iloc[0]['R²'],
    'validation_wmape': results_df.iloc[0]['WMAPE'],
    'training_date': pd.Timestamp.now(),
    'combo_means': combo_means_full if best_model_name not in ['LightGBM Vanilla', 'XGBoost', 'Random Forest'] else None,
    'metadata': metadata
}

# Salvar artefatos do modelo
with open('../data/trained_model.pkl', 'wb') as f:
    pickle.dump(model_artifacts, f)

print('💾 Modelo e artefatos salvos em: data/trained_model.pkl')
print('🎯 Pronto para gerar predições para o período de teste!')